# Planet API Python Client Introduction


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Data API is used here to check for PlanetScope coverage in a known area of interest during a known time range.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Data API Documentation](https://docs.planet.com/develop/apis/data/)

This tutorial will cover the basic operations possible with the Python client interacting with the Data API.

The basic workflow for interacting with the Data API is:
1. Search item types based on filters
2. Activate assets
3. Download assets

Additional features like saving searches and getting summary statistics are included at the end of the tutorial.

____

## Set up

In order to interact with the Planet API using the Python client, we need to import the necessary packages, authenticate our Planet account credentials, and define helper functions.

### Imports

In [1]:
import json
import os

from datetime import datetime
from planet import Auth, Planet, Session, data_filter 

# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [2]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
if not auth.is_initialized():
    auth.user_login(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

_____

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

#### Define Area of Interest

There is a provided Geojson of San Francisco to use for this example. We will read a GeoJSON's geometry into a variable so we can use it during testing. 
<br><br>
You can create your own custom geometry with a free tool [geojson.io](https://geojson.io/) instead if you wish.

In [3]:
geojson_path = os.path.join(os.getcwd(), "data", "san-francisco.json")

with open(geojson_path) as f:
    geom = json.loads(f.read())

In [4]:
geom

{'type': 'Polygon',
 'coordinates': [[[-122.47455596923828, 37.810326435534755],
   [-122.49172210693358, 37.795406713958236],
   [-122.52056121826172, 37.784282779035216],
   [-122.51953124999999, 37.6971326434885],
   [-122.38941192626953, 37.69441603823106],
   [-122.38872528076173, 37.705010235842614],
   [-122.36228942871092, 37.70935613533687],
   [-122.34992980957031, 37.727280276860036],
   [-122.37773895263672, 37.76230130281876],
   [-122.38494873046875, 37.794592824285104],
   [-122.40554809570311, 37.813310018173155],
   [-122.46150970458983, 37.805715207044685],
   [-122.47455596923828, 37.810326435534755]]]}

### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API.
- Here we are searching for "PSScene" item types with `and_filter` that are:
    - `clear_percent` of 90 or more from cloud, haze, shadow, or snow.
    - `acquired` after January 1st, 2017
    - `cloud_cover` value of less than 0.1


In [5]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

geom_filter = data_filter.geometry_filter(geom)
clear_percent_filter = data_filter.range_filter('clear_percent', 90)
date_range_filter = data_filter.date_range_filter("acquired", gt=datetime(month=1, day=1, year=2017))
cloud_cover_filter = data_filter.range_filter('cloud_cover', None, 0.1)

combined_filter = data_filter.and_filter([geom_filter, clear_percent_filter, date_range_filter, cloud_cover_filter])

In [6]:
combined_filter

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-122.47455596923828, 37.810326435534755],
      [-122.49172210693358, 37.795406713958236],
      [-122.52056121826172, 37.784282779035216],
      [-122.51953124999999, 37.6971326434885],
      [-122.38941192626953, 37.69441603823106],
      [-122.38872528076173, 37.705010235842614],
      [-122.36228942871092, 37.70935613533687],
      [-122.34992980957031, 37.727280276860036],
      [-122.37773895263672, 37.76230130281876],
      [-122.38494873046875, 37.794592824285104],
      [-122.40554809570311, 37.813310018173155],
      [-122.46150970458983, 37.805715207044685],
      [-122.47455596923828, 37.810326435534755]]]}},
  {'type': 'RangeFilter', 'field_name': 'clear_percent', 'config': {'gt': 90}},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gt': '2017-01-01T00:00:00Z'}},
  {'type': 'RangeFilter', 'field_name': '

Now let's build the request block using `create_search`:

In [7]:
search_request = pl.data.create_search(item_types=item_types, search_filter=combined_filter, name='planet_sdk_demo')

In [8]:
search_request

{'__daily_email_enabled': False,
 '_links': {'_self': 'https://api.planet.com/data/v1/searches/83bea55b4bed4afd89c6254942a51212',
  'results': 'https://api.planet.com/data/v1/searches/83bea55b4bed4afd89c6254942a51212/results'},
 'created': '2025-10-02T17:22:30.520571Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    

Then, we run the search that we just built using its `id`. We will be limiting the results to `50` for this example.

In [9]:
# Search the Data API

# The limit paramter allows us to limit the number of results from our search that are returned.
# The default limit is 100.

search_id = search_request['id']
limit = 50
item_list = pl.data.run_search(search_id=search_id, limit=limit)
item_list = list(item_list)

Now, we can iterate through our search results:

In [10]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

20250927_192606_07_253d PSScene
20250926_192540_31_2527 PSScene
20250926_192538_01_2527 PSScene
20250926_192326_27_2531 PSScene
20250926_192323_96_2531 PSScene
20250923_192443_55_24b8 PSScene
20250923_192445_87_24b8 PSScene
20250923_192323_44_24fa PSScene
20250923_192321_28_24fa PSScene
20250919_191703_17_250a PSScene
20250919_191700_97_250a PSScene
20250917_192935_26_24df PSScene
20250917_192200_03_24d1 PSScene
20250916_191655_04_2515 PSScene
20250914_192352_64_24de PSScene
20250907_192420_83_24e5 PSScene
20250907_192236_33_254a PSScene
20250907_192238_65_254a PSScene
20250906_193017_20_24fb PSScene
20250906_193015_12_24fb PSScene
20250902_192247_46_252b PSScene
20250831_192025_20_24dc PSScene
20250831_192027_39_24dc PSScene
20250830_192421_93_2526 PSScene
20250830_192547_51_2530 PSScene
20250829_192734_79_24df PSScene
20250829_192216_98_2527 PSScene
20250829_192736_86_24df PSScene
20250829_192646_07_24ed PSScene
20250829_192648_13_24ed PSScene
20250828_192712_72_252b PSScene
20250828

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

_____

## Assets and downloads

After a search returns results, the Python client can be used to check for assets and initiate downloads. Let's start by looking at one item and the assets available to download for that item.

For more information on Items and Assets, check out [Items & Assets](https://developers.planet.com/docs/apis/data/items-assets/) on the Planet Developer Center.

In [11]:
item_list[0]['properties']

{'acquired': '2025-09-27T19:26:06.070563Z',
 'anomalous_pixels': 0,
 'clear_confidence_percent': 94,
 'clear_percent': 99,
 'cloud_cover': 0,
 'cloud_percent': 0,
 'ground_control': True,
 'gsd': 3.8,
 'heavy_haze_percent': 0,
 'instrument': 'PSB.SD',
 'item_type': 'PSScene',
 'light_haze_percent': 0,
 'pixel_resolution': 3,
 'provider': 'planetscope',
 'published': '2025-09-28T04:26:12Z',
 'publishing_stage': 'finalized',
 'quality_category': 'test',
 'satellite_azimuth': 277.7,
 'satellite_id': '253d',
 'shadow_percent': 0,
 'snow_ice_percent': 1,
 'strip_id': '8335954',
 'sun_azimuth': 166.4,
 'sun_elevation': 49.5,
 'updated': '2025-09-28T12:58:31Z',
 'view_angle': 4.5,
 'visible_confidence_percent': 63,
 'visible_percent': 100}

In [12]:
# As an example, let's look at the first result in our item_list and grab the item_id and item_type:
item = item_list[0]

item_id = item['id']
item_type = item['properties']['item_type']

print(item_id, item_type)

20250927_192606_07_253d PSScene


In [13]:
# The list of assets for an item that a user has access to can be retrieved with `permissions`
item['_permissions']

['assets.ortho_visual:download',
 'assets.ortho_analytic_4b:download',
 'assets.ortho_analytic_4b_xml:download',
 'assets.basic_analytic_4b:download',
 'assets.basic_analytic_4b_xml:download',
 'assets.basic_analytic_4b_rpc:download',
 'assets.ortho_analytic_4b_sr:download',
 'assets.ortho_udm2:download',
 'assets.basic_udm2:download',
 'assets.ortho_analytic_8b:download',
 'assets.ortho_analytic_8b_xml:download',
 'assets.basic_analytic_8b:download',
 'assets.basic_analytic_8b_xml:download',
 'assets.ortho_analytic_8b_sr:download']

There are a few steps involved in order to download an asset using the Planet Python Client:

* **Get Asset:** Get a description of our asset based on the specifications we're looking for
* **Activate Asset:** Activate the asset with the given description
* **Wait Asset:** Wait for the asset to be activated
* **Download Asset:** Now our asset is ready for download!

Let's go through these steps below. We'll do this for our analytic asset, as well as the analytic_xml asset.

In [14]:
# Analytic Asset
asset_type_id = 'ortho_analytic_4b'
asset_desc = pl.data.get_asset(item_type_id=item_type, item_id=item_id, asset_type_id=asset_type_id)

pl.data.activate_asset(asset_desc)
print(f'{item_id} activated')

print(f'Awaiting {asset_type_id} activation')
pl.data.wait_asset(asset_desc)

asset_path = pl.data.download_asset(asset=asset_desc, directory='output', overwrite=True)
print(f'{item_id} downloaded to: {asset_path}')

20250927_192606_07_253d activated
Awaiting ortho_analytic_4b activation


output/20250927_192606_07_253d_3B_AnalyticMS.tif: 100%|███████| 520k/520k [00:10<00:00, 52.2MB/s]

20250927_192606_07_253d downloaded to: output/20250927_192606_07_253d_3B_AnalyticMS.tif


In [15]:
# Analytic XML Asset
asset_type_id = 'ortho_analytic_4b_xml'
asset_desc = pl.data.get_asset(item_type_id=item_type, item_id=item_id, asset_type_id=asset_type_id)

pl.data.activate_asset(asset_desc)
print(f'{item_id} activated')

print(f'Awaiting {asset_type_id} activation')
pl.data.wait_asset(asset_desc)

asset_path = pl.data.download_asset(asset=asset_desc, directory='output', overwrite=True)
print(f'{item_id} downloaded to: {asset_path}')

20250927_192606_07_253d activated
Awaiting ortho_analytic_4b_xml activation


output/20250927_192606_07_253d_3B_AnalyticMS_metadata.xml: 100%|█| 0.01k/0.01k [00:00<00:00, 21.0

20250927_192606_07_253d downloaded to: output/20250927_192606_07_253d_3B_AnalyticMS_metadata.xml


Congratulations! Both the `ortho_analytic_4b` and `ortho_analytic_4b_xml` assets should be saved in our `output` directory.

_______

# Other Data-API Features

## Saved Searches

The Data API client can also help in managing saved searches on the Planet Platform.

In [16]:
data_searches = [search for search in pl.data.list_searches()]

# We may have a lot of saved searches!
print(f'number of searches: {len(data_searches)}')

number of searches: 28


View your saved searches. Here we're viewing our first 50 results.

In [ ]:
for search in data_searches[:50]:
    print(search['id'], search['name'])

We can find the saved search we're looking for by filtering on our search id:

In [18]:
search = pl.data.get_search(data_searches[0]['id'])

In [23]:
search

{'__daily_email_enabled': False,
 '_links': {'_self': '<redacted>', 'results': '<redacted>'},
 'created': '2025-10-01T18:27:48.527670Z',
 'filter': {'config': [{'config': {'coordinates': [[[-122.47455596923828,
        37.810326435534755],
       [-122.49172210693358, 37.795406713958236],
       [-122.52056121826172, 37.784282779035216],
       [-122.51953124999999, 37.6971326434885],
       [-122.38941192626953, 37.69441603823106],
       [-122.38872528076173, 37.705010235842614],
       [-122.36228942871092, 37.70935613533687],
       [-122.34992980957031, 37.727280276860036],
       [-122.37773895263672, 37.76230130281876],
       [-122.38494873046875, 37.794592824285104],
       [-122.40554809570311, 37.813310018173155],
       [-122.46150970458983, 37.805715207044685],
       [-122.47455596923828, 37.810326435534755]]],
     'type': 'Polygon'},
    'field_name': 'geometry',
    'type': 'GeometryFilter'},
   {'config': {'gt': 90.0},
    'field_name': 'clear_percent',
    'type': 'R

# Statistics

The Data API client can also help report statistical summaries of the amount of data in the Planet API.

In [20]:
stats = pl.data.get_stats(interval='year', search_filter=combined_filter, item_types=item_types)

In [21]:
indent(stats)

{
  "buckets": [
    {
      "count": 374,
      "start_time": "2017-01-01T00:00:00.000000Z"
    },
    {
      "count": 475,
      "start_time": "2018-01-01T00:00:00.000000Z"
    },
    {
      "count": 624,
      "start_time": "2019-01-01T00:00:00.000000Z"
    },
    {
      "count": 742,
      "start_time": "2020-01-01T00:00:00.000000Z"
    },
    {
      "count": 659,
      "start_time": "2021-01-01T00:00:00.000000Z"
    },
    {
      "count": 450,
      "start_time": "2022-01-01T00:00:00.000000Z"
    },
    {
      "count": 297,
      "start_time": "2023-01-01T00:00:00.000000Z"
    },
    {
      "count": 421,
      "start_time": "2024-01-01T00:00:00.000000Z"
    },
    {
      "count": 274,
      "start_time": "2025-01-01T00:00:00.000000Z"
    }
  ],
  "interval": "year",
  "utc_offset": "+0h"
}
